In [1]:
# From here: https://stackoverflow.com/questions/49264194/import-py-file-in-another-directory-in-jupyter-notebook
import sys
import os

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
import numpy as np
import copy

import src.mcmc as mcmc
import src.utils as utils
from src.ansatz import RBM
from src.model import Model
from config_parser import get_config_file

import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
visible_layer_size = 3
hidden_layer_size = 8
seed = 44  # Seed for random number generator
walker_steps = 1000
off_diag_size = 1



In [4]:
np.random.seed(seed)

b = utils.random_complex_array(visible_layer_size)  # Visible layer bias
c = utils.random_complex_array(hidden_layer_size)  # Hidden layer bias
W = utils.random_complex_matrix(visible_layer_size, hidden_layer_size)  # Visible - hidden weights
H_full = utils.random_hamiltonian(2**visible_layer_size)  # Hamiltonian
walker_full = mcmc.Walker(steps=walker_steps)
rbm_full = RBM(visible_bias=b, hidden_bias=c, weights=W)  # Initializing RBM currently with random configuration and parameters
model_full = Model(rbm_full, walker_full, H_full)  # Initializing model with RBM and Hamiltonian

In [5]:
H_sparse = utils.random_diagonal_hamiltonian(2*visible_layer_size, off_diag_size)
walker_sparse = mcmc.Walker(steps=walker_steps)
rbm_sparse = RBM(visible_bias=b, hidden_bias=c, weights=W)
model_sparse = Model(rbm_sparse, walker_sparse, H_sparse)

In [ ]:
# Training models

termination_condition = 1e-5
gradient_steps = 100
learning_rate = 0.05

In [6]:
fd_plot_full_list = model_full.gradient_descent(gradient_method='finite_difference', 
                                              exact_dist=True, n_steps=gradient_steps, 
                                              learning_rate=learning_rate, 
                                              termination_condition=termination_condition)

Gradient descent step 1, energy: -2.7841418862137894
Gradient descent step 2, energy: -3.4222224366782044
Gradient descent step 3, energy: -3.45220836496278
Gradient descent step 4, energy: -3.8940726297657298
Gradient descent step 5, energy: -4.214729035683401
Gradient descent step 6, energy: -4.379317067604184
Gradient descent step 7, energy: -4.54756343182437
Gradient descent step 8, energy: -4.797966852382935
Gradient descent step 9, energy: -5.020366808076551
Gradient descent step 10, energy: -4.951601886321749
Gradient descent step 11, energy: -4.607017753939354
Gradient descent step 12, energy: -4.291527644533372
Gradient descent step 13, energy: -4.096916606948763
Gradient descent step 14, energy: -3.9892666691119754
Gradient descent step 15, energy: -3.933404846802686
Gradient descent step 16, energy: -3.906045280234991
Gradient descent step 17, energy: -3.878655448662445
Gradient descent step 18, energy: -3.807465132063589
Gradient descent step 19, energy: -3.647524060075298


IndexError: index 6 is out of bounds for axis 1 with size 6

In [7]:
fd_plot_sparse_list = model_sparse.gradient_descent(gradient_method='finite_difference', 
                                              exact_dist=True, n_steps=gradient_steps, 
                                              learning_rate=learning_rate, 
                                              termination_condition=termination_condition)

IndexError: index 6 is out of bounds for axis 1 with size 6

In [ ]:
print(f"Full Hamiltonian time: {model_full.optimizing_time}")
print(f"Sparse Hamiltonian time: {model_sparse.optimizing_time}")

In [ ]:
plt.plot(np.real(fd_plot_full_list), label='Full H')
plt.plot(np.real(fd_plot_sparse_list), label='Sparse H')

plt.axhline(y=min(np.linalg.eigvalsh(H_full)), color='red', linestyle='--', label='Ground State full')
plt.axhline(y=min(np.linalg.eigvalsh(H_sparse)), color='blue', linestyle='--', label='Ground State sparse')
plt.title('Full vs Sparse H gradent descent')
plt.xlabel('Steps')
plt.ylabel('Energy')
plt.legend()
